<a href="https://colab.research.google.com/github/ajitjaison/Datathons/blob/master/Datathon4_Sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
%matplotlib inline

In [ ]:
concrete_data  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/concrete_data/training_set_label.csv" )
concrete_data.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,298.20,0.0,107.00,209.70,11.10,879.6,744.20,28,31.875165
1,397.00,0.0,0.00,186.00,0.00,1040.0,734.00,28,36.935229
2,251.37,0.0,118.27,188.45,6.35,1028.4,757.73,56,36.638755
3,304.00,140.0,0.00,214.00,6.00,895.0,722.00,28,33.418902
4,297.00,0.0,0.00,186.00,0.00,1040.0,734.00,7,30.957472


In [ ]:
data = concrete_data.rename(columns={'Cement (component 1)(kg in a m^3 mixture)':"cement",
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)':"furnace_slag",
       'Fly Ash (component 3)(kg in a m^3 mixture)':"fly_ash",
       'Water  (component 4)(kg in a m^3 mixture)':"water",
       'Superplasticizer (component 5)(kg in a m^3 mixture)':"super_plasticizer",
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)':"coarse_agg",
       'Fine Aggregate (component 7)(kg in a m^3 mixture)':"fine_agg", 'Age (day)':"age",
       'Concrete compressive strength(MPa, megapascals) ':"compressive_strength"})
data.head()

,cement,furnace_slag,fly_ash,water,super_plasticizer,coarse_agg,fine_agg,age,compressive_strength
0,298.20,0.0,107.00,209.70,11.10,879.6,744.20,28,31.875165
1,397.00,0.0,0.00,186.00,0.00,1040.0,734.00,28,36.935229
2,251.37,0.0,118.27,188.45,6.35,1028.4,757.73,56,36.638755
3,304.00,140.0,0.00,214.00,6.00,895.0,722.00,28,33.418902
4,297.00,0.0,0.00,186.00,0.00,1040.0,734.00,7,30.957472


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cement                772 non-null    float64
 1   furnace_slag          772 non-null    float64
 2   fly_ash               772 non-null    float64
 3   water                 772 non-null    float64
 4   super_plasticizer     772 non-null    float64
 5   coarse_agg            772 non-null    float64
 6   fine_agg              772 non-null    float64
 7   age                   772 non-null    int64  
 8   compressive_strength  772 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 54.4 KB


In [ ]:
print (data.isnull().sum())

cement                  0
furnace_slag            0
fly_ash                 0
water                   0
super_plasticizer       0
coarse_agg              0
fine_agg                0
age                     0
compressive_strength    0
dtype: int64


In [ ]:
print("Number of rows    :",data.shape[0])
print("Number of columns :",data.shape[1])

Number of rows    : 772
Number of columns : 9


In [ ]:
from sklearn.preprocessing import MaxAbsScaler
#scaler = MaxAbsScaler()
#data.age = scaler.fit_transform(data.age.values.reshape(-1, 1))
data.head()
# pure water-cement ratio feature
input_features_unscaled_with_wcratio = data.copy()
input_features_unscaled_with_wcratio.insert(input_features_unscaled_with_wcratio.shape[-1]-1,'water-cement-ratio', input_features_unscaled_with_wcratio['water']/input_features_unscaled_with_wcratio['cement'])
input_features_unscaled_with_wcratio.drop(['water', 'cement'], axis=1, inplace=True)

# scale it!
scaler = MaxAbsScaler()
input_features_unscaled_with_wcratio_scaled = input_features_unscaled_with_wcratio.copy()
input_features_unscaled_with_wcratio_sc = scaler.fit_transform(input_features_unscaled_with_wcratio)
input_features_unscaled_with_wcratio_scaled.loc[:,:] = input_features_unscaled_with_wcratio_sc

# making the scaling function more accessible
extract_scaling_function = np.ones((1,input_features_unscaled_with_wcratio.shape[1]))
extract_scaling_function = scaler.inverse_transform(extract_scaling_function)

new_data = input_features_unscaled_with_wcratio_scaled.copy()


In [ ]:
new_data.head()

,furnace_slag,fly_ash,super_plasticizer,coarse_agg,fine_agg,age,water-cement-ratio,compressive_strength
0,0.000000,0.534733,0.344720,0.768210,0.749748,0.076712,0.373585,0.385902
1,0.000000,0.000000,0.000000,0.908297,0.739472,0.076712,0.248898,0.447162
2,0.000000,0.591054,0.197205,0.898166,0.763379,0.153425,0.398274,0.443573
3,0.389538,0.000000,0.186335,0.781659,0.727383,0.076712,0.373972,0.404591
4,0.000000,0.000000,0.000000,0.908297,0.739472,0.019178,0.332702,0.374791


In [ ]:
check_data = extract_scaling_function * new_data
check_data

,furnace_slag,fly_ash,super_plasticizer,coarse_agg,fine_agg,age,water-cement-ratio,compressive_strength
0,0.0,107.00,11.10,879.60,744.20,28.0,0.703219,31.875165
1,0.0,0.00,0.00,1040.00,734.00,28.0,0.468514,36.935229
2,0.0,118.27,6.35,1028.40,757.73,56.0,0.749692,36.638755
3,140.0,0.00,6.00,895.00,722.00,28.0,0.703947,33.418902
4,0.0,0.00,0.00,1040.00,734.00,7.0,0.626263,30.957472
...,...,...,...,...,...,...,...,...
767,0.0,0.00,0.00,1111.60,784.30,7.0,0.735446,11.483912
768,0.0,98.75,12.80,987.76,889.01,56.0,0.634725,42.030457
769,0.0,0.00,0.00,889.80,945.00,3.0,0.752941,8.204075
770,0.0,125.40,7.77,1090.00,804.01,100.0,0.850325,40.568768


In [ ]:
X = new_data.drop(['compressive_strength'], axis = 1)
y = new_data['compressive_strength']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3, random_state = 123)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
pred_rf = rf.predict(X_val)

In [ ]:
from sklearn.metrics import mean_squared_error
print('Root Mean Squared Error is: ', np.sqrt(mean_squared_error(y_val, pred_rf))) 

Root Mean Squared Error is:  0.06657537735873756


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
pred_gbr = gbr.predict(X_val)
from sklearn.metrics import mean_squared_error
print('Root Mean Squared Error is: ', np.sqrt(mean_squared_error(y_val, pred_gbr))) 

Root Mean Squared Error is:  5.478900706687716


In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
xgr =XGBRegressor()
xgr.fit(X_train,y_train)

[04:26:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
pred_xgr = xgr.predict(X_val)
from sklearn.metrics import mean_squared_error
print('Root Mean Squared Error is: ', np.sqrt(mean_squared_error(y_val, pred_xgr))) 

Root Mean Squared Error is:  0.06578959890963229


In [ ]:
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/concrete_data/testing_set_label.csv')
test_data.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day)
0,167.00,187.00,195.00,185.00,7.00,898.0,636.00,28
1,148.50,139.40,108.60,192.70,6.10,892.4,780.00,28
2,154.80,0.00,142.80,193.30,9.10,877.2,867.70,28
3,236.00,0.00,0.00,193.00,0.00,968.0,885.00,365
4,168.88,42.22,124.25,158.33,10.83,1080.8,796.15,14


In [ ]:
test_data_1 = test_data.rename(columns={'Cement (component 1)(kg in a m^3 mixture)':"cement",
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)':"furnace_slag",
       'Fly Ash (component 3)(kg in a m^3 mixture)':"fly_ash",
       'Water  (component 4)(kg in a m^3 mixture)':"water",
       'Superplasticizer (component 5)(kg in a m^3 mixture)':"super_plasticizer",
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)':"coarse_agg",
       'Fine Aggregate (component 7)(kg in a m^3 mixture)':"fine_agg", 'Age (day)':"age",
       'Concrete compressive strength(MPa, megapascals) ':"compressive_strength"})
test_data_1.head()

,cement,furnace_slag,fly_ash,water,super_plasticizer,coarse_agg,fine_agg,age
0,167.00,187.00,195.00,185.00,7.00,898.0,636.00,28
1,148.50,139.40,108.60,192.70,6.10,892.4,780.00,28
2,154.80,0.00,142.80,193.30,9.10,877.2,867.70,28
3,236.00,0.00,0.00,193.00,0.00,968.0,885.00,365
4,168.88,42.22,124.25,158.33,10.83,1080.8,796.15,14


In [ ]:
T1 = test_data_1.copy()
T1.insert(T1.shape[-1],'water-cement-ratio', T1['water']/T1['cement'])
T1.drop(['water', 'cement'], axis=1, inplace=True)

# scale it!
scaler = MaxAbsScaler()
T1_scaled = T1.copy()
T1_sc = scaler.fit_transform(T1)
T1_scaled.loc[:,:] = T1_sc

# making the scaling function more accessible
extract_scaling_f = np.ones((1,T1.shape[1]))
extract_scaling_f = scaler.inverse_transform(extract_scaling_f)

new_data1 = T1_scaled.copy()
new_data1.head()

,furnace_slag,fly_ash,super_plasticizer,coarse_agg,fine_agg,age,water-cement-ratio
0,0.612512,0.97500,0.217391,0.791678,0.640741,0.076712,0.588510
1,0.456600,0.54300,0.189441,0.786741,0.785815,0.076712,0.689373
2,0.000000,0.71400,0.282609,0.773340,0.874169,0.076712,0.663376
3,0.000000,0.00000,0.000000,0.853390,0.891598,1.000000,0.434454
4,0.138290,0.62125,0.336335,0.952834,0.802085,0.038356,0.498063


In [ ]:
xgr.feature_importances_

array([0.08241802, 0.03451257, 0.1236321 , 0.02132194, 0.04576763,
       0.37207282, 0.3202748 ], dtype=float32)

In [ ]:
extract_scaling_f.shape

(1, 7)

In [ ]:
target = xgr.predict(new_data1)

In [ ]:
target

array([0.3479968 , 0.37386656, 0.18242404, 0.40184134, 0.36939958,
       0.5763907 , 0.2785781 , 0.48522434, 0.65482265, 0.58104247,
       0.2516744 , 0.79039025, 0.6403819 , 0.16160709, 0.289981  ,
       0.40504578, 0.15041557, 0.31793797, 0.4739764 , 0.2730748 ,
       0.2702497 , 0.43272763, 0.791437  , 0.37440807, 0.5217613 ,
       0.9391786 , 0.6099823 , 0.480228  , 0.5520355 , 0.87589777,
       0.45893055, 0.4745664 , 0.5223409 , 0.37438405, 0.28008965,
       0.42452455, 0.41640455, 0.21306238, 0.26267847, 0.50302017,
       0.8174261 , 0.6119052 , 0.20498651, 0.31451905, 0.4950422 ,
       0.36749056, 0.8149291 , 0.4957741 , 0.42115384, 0.3707478 ,
       0.7526096 , 0.37564525, 0.41547042, 0.52087504, 0.17773661,
       0.3488318 , 0.54532444, 0.45445693, 0.3595397 , 0.14562678,
       0.44363928, 0.2702497 , 0.43587655, 0.34943762, 0.6144894 ,
       0.38129434, 0.5886418 , 0.6163554 , 0.41344225, 0.10042545,
       0.11161658, 0.1569969 , 0.3104493 , 0.5308513 , 0.41077

In [ ]:
extract_scaling_function

array([[ 359.4       ,  200.1       ,   32.2       , 1145.        ,
         992.6       ,  365.        ,    1.88235294,   82.5992248 ]])

In [ ]:
target_new = extract_scaling_function[0][7] * target
target_new

array([28.744267 , 30.881088 , 15.068085 , 33.191784 , 30.51212  ,
       47.609425 , 23.010336 , 40.079155 , 54.087845 , 47.99366  ,
       20.788113 , 65.28562  , 52.89505  , 13.34862  , 23.952208 ,
       33.456467 , 12.42421  , 26.26143  , 39.150085 , 22.555769 ,
       22.322416 , 35.74297  , 65.372086 , 30.925817 , 43.09708  ,
       77.575424 , 50.384068 , 39.666462 , 45.597706 , 72.34848  ,
       37.90731  , 39.19882  , 43.144955 , 30.923834 , 23.13519  ,
       35.0654   , 34.394695 , 17.598787 , 21.697039 , 41.549076 ,
       67.51876  , 50.5429   , 16.931728 , 25.97903  , 40.890102 ,
       30.354437 , 67.312515 , 40.950558 , 34.786983 , 30.623482 ,
       62.164974 , 31.028008 , 34.317535 , 43.023876 , 14.680906 ,
       28.813238 , 45.043377 , 37.537792 , 29.6977   , 12.02866  ,
       36.64426  , 22.322416 , 36.003067 , 28.863277 , 50.756348 ,
       31.494617 , 48.62136  , 50.91048  , 34.15001  ,  8.295065 ,
        9.219443 , 12.967823 , 25.642872 , 43.847908 , 33.9297

In [ ]:
res = pd.DataFrame(target_new) # target are nothing but the final predictions of your model on input features of your new unseen test data
res.index = test_data.index # its important for comparison. Here "test_new" is your new test dataset
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('submission.csv')         
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>